In [4]:
# 2-1 LLM and Chat Models
from langchain_openai import OpenAI # LLM
from langchain.chat_models import ChatOpenAI # Chat model

llm = OpenAI()
chat = ChatOpenAI()

a = llm.predict("How many planets are in the solar system?")
b = chat.predict("How many planets are in the solar system?")

a, b

('\n\nThere are 8 planets in the solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.',
 'There are 8 planets in the solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.')

In [10]:
## 2-2. Predict Messages
from langchain.chat_models import ChatOpenAI
# HumanMessage - 인간이 작성하는 Message
# AIMessage - AI에 의해서 보내지는 Message
# SystemMessage - LLM에 설정들을 제공하기 위한 Message
from langchain.schema import HumanMessage, AIMessage, SystemMessage

chat = ChatOpenAI(
    temperature=0.1, # 모델의 창의성을 조절하는 옵션 (높을 수록 창의적임)
)

messages = [
    SystemMessage(content="You are a geography expert. And you only reply in Italian."),
    AIMessage(content="Ciao, mi chiamo Paolo!"),
    HumanMessage(content="What is the distance between the Mexico and Thailand. Also, what is your name?"),
]

chat.invoke(messages)

AIMessage(content='Ciao! La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri. Come posso aiutarti oggi?', response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 53, 'total_tokens': 87}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5d70edf0-b4f7-4009-b92d-67c8026a8622-0')

In [3]:
## 2-3. Prompt Templates
from langchain.chat_models import ChatOpenAI
# PromptTemplate - 문자열을 이용한 template 생성
# ChatPromptTemplate - message를 이용하여 template 생성
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate

chat = ChatOpenAI(
    temperature=0.1, # 모델의 창의성을 조절하는 옵션 (높을 수록 창의적임)
)

# 일반 문자열 출력 예제
# template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}")
# prompt = template.format(country_a="Mexico", country_b="Thailand")
# chat.predict(prompt)

# 메시지 출력 예제
messages = [
    SystemMessagePromptTemplate.from_template("You are a geography expert. And you only reply in {language}."),
    AIMessagePromptTemplate.from_template("Ciao, mi chiamo {name}!"),
    HumanMessagePromptTemplate.from_template("What is the distance between the {country_a} and {country_b}. Also, what is your name?")
]

template = ChatPromptTemplate.from_messages(messages)

prompt = template.format_messages(language="Italian", name="Paolo", country_a="Mexico", country_b="Thailand")

chat.invoke(prompt)

AIMessage(content='Ciao! La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri. Come posso aiutarti oggi?', response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 53, 'total_tokens': 87}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-dfa458b7-47b5-4a38-bb5c-5915b03889c9-0')

In [7]:
# 2-4. OutputParser and LCEL
from langchain.chat_models import ChatOpenAI
from langchain.schema import BaseOutputParser
# PromptTemplate - 문자열을 이용한 template 생성
# ChatPromptTemplate - message를 이용하여 template 생성
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate

chat = ChatOpenAI(
    temperature=0.1, # 모델의 창의성을 조절하는 옵션 (높을 수록 창의적임)
)

# 문자열 출력을 파싱하는 BaseOutputParser 확장하는 커스텀 OutputParser
class CommaOutputParser(BaseOutputParser):
    def parse(self, text: str) -> str:
        items = text.strip().split(",")
        return list(map(str.strip, items))
    

# OutputParser 예제 (LCEL 적용 전)
# p = CommaOutputParser()
# messages = [
#     SystemMessagePromptTemplate.from_template("You are a list gernerating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do Not reply with else."),
#     HumanMessagePromptTemplate.from_template("{question}")
# ]
# template = ChatPromptTemplate.from_messages(messages)
# prompt = template.format_messages(max_items=10, question="What are the colors?")
# res = chat.invoke(prompt)
# p.parse(res.content)

# OutputParser 예제 (LCEL 적용 후)
messages = [
    SystemMessagePromptTemplate.from_template("You are a list gernerating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do Not reply with else."),
    HumanMessagePromptTemplate.from_template("{question}")
]
template = ChatPromptTemplate.from_messages(messages)

chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items":10, 
    "question":"What are the colors?",
})

['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [ ]:
# 2-5. Chaining Chains